In [4]:
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, Var

In [5]:
data = pd.read_excel("Exercise4_Location_Planning_Data.xlsx",sheet_name="Tabelle1", header=0)

data2 = pd.read_excel("Exercise4_Location_Planning_Data.xlsx",sheet_name="Tabelle2", header=0)
data2

,fi,ai
0,689,103
1,723,101
2,855,76
3,350,106
4,699,62
5,361,109
6,552,74
7,443,115


In [6]:
C = np.array(data.loc[:,"Location 1":"Location 8"]).T
b = np.array(data.loc[:,"bj"])

a = np.array(data2.loc[:,"ai"])
f = np.array(data2.loc[:,"fi"])

In [7]:
m = Model("WLP")

X = m.addMVar(C.shape, vtype = GRB.CONTINUOUS, lb=float(0), ub=float('inf'), name = "X")
y = m.addMVar(shape = (8), vtype = GRB.BINARY, name="y")
m.update()

m.addConstrs(X[:,j]@np.ones(8) == b[j] for j in range(10)) # Meet demand b_j

m.addConstrs(X[i,:]@np.ones(10) <= a[i]*y[i] for i in range(8)) # Demand restriction

m.addConstr(sum(y) <= 3) # Max 3 Locations
m.addConstr(y[4]+y[5] <= 1) # 3 and 4 mutually exclusive
m.update()

m.setObjective(sum(C[i,j]*X[i,j] for i in range(8) for j in range(10)) + f@y)
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-29
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 20 rows, 88 columns and 178 nonzeros
Model fingerprint: 0x8a761b00
Variable types: 80 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+01, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve time: 0.00s
Presolved: 20 rows, 88 columns, 178 nonzeros
Variable types: 80 continuous, 8 integer (8 binary)
Found heuristic solution: objective 8429.0000000

Root relaxation: objective 6.074463e+03, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 6074.46279    0    7 8429.00000 6074.46279  27.9%     -  

In [8]:
np.array(m.getAttr('X')[:80]).reshape(C.shape)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 40.,  0.,  0.,  0.,  0., 16.,  0., 20.,  0.],
       [10.,  0., 11.,  0., 46., 13.,  0., 18.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 39.,  0.,  0.,  2., 42.,  0., 26.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [9]:
print("X:")
print(np.array(m.getAttr("X")[:80]).reshape(C.shape))
print("y:")
print(np.array(m.getAttr("X")[80:]).reshape(a.shape))
print("optimal Value:")
print(m.objVal)

X:
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 40.  0.  0.  0.  0. 16.  0. 20.  0.]
 [10.  0. 11.  0. 46. 13.  0. 18.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 39.  0.  0.  2. 42.  0. 26.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
y:
[-0. -0.  1.  1.  0.  1. -0. -0.]
optimal Value:
7739.0
